In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
!pip install tweet-preprocessor
!pip install transformers
!pip install sentence_transformers
!pip install ranky
!pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Imports

In [5]:
import preprocessor as p
import re
import nltk
import numpy as np
import pandas as pd
import transformers
import torch
import string
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
# from py_rankaggregation import CombANZ
import ranky as rk
from keybert import KeyBERT
import spacy
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Preprocessing

In [8]:
# download the necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
# Preprocessing
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Removing URLs, retweets, mentions, stopwords
    rtwords = ['rt', 'RT', 'retweet', 'Retweet', 'RETWEET']
    stops = stopwords.words('english') + rtwords
    
    mess = re.sub(r"http\S+", "", mess)
    mess = re.sub("@[A-Za-z0-9_]+","", mess)
    mess = re.sub("#","", mess)

    # Define a regular expression pattern that matches English words
    pattern = re.compile(r'[a-zA-Z]+')
    
    # Find all matches of the pattern in the text
    matches = re.findall(pattern, mess)
    
    # Join the matches back into a single string
    english_text = ' '.join(matches)
    
    # # STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
    # # Check characters to see if they are in punctuation
    nopunc = [char for char in english_text if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # # Now just remove any stopwords
    # return ' '.join([word for word in nopunc.split() if word.lower() not in stops])
    
    # define the tokenizer, stemmer, and lemmatizer
    stop_words = set(stopwords.words('english'))
    stemmer = SnowballStemmer('english')
    lemmatizer = WordNetLemmatizer()

    # convert to lowercase
    tweet = nopunc.lower()
    # tokenize the tweet
    # tokens = word_tokenize(tweet)
    # remove stop words
    # tokens = [token for token in tokens if token not in stop_words]
    # # stem the tokens
    # tokens = [stemmer.stem(token) for token in tokens]
    # # lemmatize the tokens
    # tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # print("here\n")
    doc = nlp(tweet)
    # doc = [token for token in doc if detect(token) == 'en']
    doc = [token for token in doc if token not in stop_words]
    
    # print("here2\n")
    # Lemmatize the tokens
    lemmas = [token.lemma_ for token in doc]
    
    # print("here3\n")
    # Join the lemmas back into a single string
    processed_tweet = ' '.join(lemmas)

    # print("here4\n")
    # # re-join the tokens into a string
    # processed_tweet = ' '.join(tokens)
    return processed_tweet

## Loading the dataset

In [10]:
Cov19_tweets = pd.read_csv('/content/drive/MyDrive/SMAI-project/data/Covid19_tweets.csv')

In [11]:
Cov19_tweets

,Date_of_tweet,User_handle,username,name,place,Tweet,language,replies_count,hashtags,sentiment
0,2021-04-27 23:55:06 UTC,23772644,bbcindonesia,BBC News Indonesia,NaN,"Varian baru Covid India sudah masuk Indonesia,...",in,15,[],NEGATIVE
1,2021-04-27 23:35:39 UTC,17128975,cnnindonesia,CNN Indonesia,NaN,Badai Covid India hingga Junta Myanmar Pikir-p...,in,0,['kilasinternasional'],NEGATIVE
2,2021-04-27 23:18:00 UTC,61377303,lintas_mnctv,Lintas_MNCTV,NaN,Peringatan WHO: Krisis Covid India Bisa Makin ...,in,0,[],NEGATIVE
3,2021-04-27 23:16:27 UTC,576348125,nagma_morarji,Nagma,NaN,German Army working around the clock to help I...,en,0,"['covidemergency2021', 'covidsecondwaveinindia...",NEGATIVE
4,2021-04-27 22:04:08 UTC,559652727,pkashwan,Prakash Kashwan प्रकाश कसवाँ,NaN,"TBC, this is an old report. The point is that ...",en,0,"['covidindiainfo', 'covidindia2021', 'covidind...",NEGATIVE
...,...,...,...,...,...,...,...,...,...,...
9650,2020-03-06 17:13:33 UTC,4888435970,dipika_bajpai,Dipika Bajpai,NaN,"#COVID OR no Covid, India is not China or USA....",en,3,['covid'],NEGATIVE
9651,2020-03-04 05:36:35 UTC,31059478,toihyderabad,TOI Hyderabad,NaN,Watch #CoronaVirusUpdate from Gandhi Hospital ...,en,1,"['coronavirusupdate', 'secunderabad', 'coronav...",NEGATIVE
9652,2020-03-03 11:14:53 UTC,31059478,toihyderabad,TOI Hyderabad,NaN,#CoronavirusUpdate || A 15 member team from #T...,en,1,"['coronavirusupdate', 'telangana', 'kerala', '...",POSITIVE
9653,2020-03-02 12:39:20 UTC,73043514,gargirawat,Gargi Rawat,NaN,Ye gads ... Now NDTV’s @harsha_ndtv reports a...,en,4,"['coronaoutbreak', 'covid', 'india']",NEGATIVE


In [12]:
Cov19_tweets = Cov19_tweets[['Date_of_tweet', 'Tweet', 'User_handle']]
Cov19_tweets

,Date_of_tweet,Tweet,User_handle
0,2021-04-27 23:55:06 UTC,"Varian baru Covid India sudah masuk Indonesia,...",23772644
1,2021-04-27 23:35:39 UTC,Badai Covid India hingga Junta Myanmar Pikir-p...,17128975
2,2021-04-27 23:18:00 UTC,Peringatan WHO: Krisis Covid India Bisa Makin ...,61377303
3,2021-04-27 23:16:27 UTC,German Army working around the clock to help I...,576348125
4,2021-04-27 22:04:08 UTC,"TBC, this is an old report. The point is that ...",559652727
...,...,...,...
9650,2020-03-06 17:13:33 UTC,"#COVID OR no Covid, India is not China or USA....",4888435970
9651,2020-03-04 05:36:35 UTC,Watch #CoronaVirusUpdate from Gandhi Hospital ...,31059478
9652,2020-03-03 11:14:53 UTC,#CoronavirusUpdate || A 15 member team from #T...,31059478
9653,2020-03-02 12:39:20 UTC,Ye gads ... Now NDTV’s @harsha_ndtv reports a...,73043514


In [13]:
raw_tweets = Cov19_tweets['Tweet'].tolist()
raw_tweets

['Varian baru Covid India sudah masuk Indonesia, dengan adanya 10 orang yang terpapar. Enam orang kasus impor dan empat orang penularan lokal di Sumatra dan Jawa Barat.   Penelurusuran kontak Indonesia lemah, kata ahli penyakit menular.   https://t.co/D5N5JgW1Ec',
 'Badai Covid India hingga Junta Myanmar Pikir-pikir Usul ASEAN #KilasInternasional  https://t.co/OwdhGFstIP',
 'Peringatan WHO: Krisis Covid India Bisa Makin Buruk  https://t.co/rPOyFAeRoc',
 'German Army working around the clock to help India with its Covid crisis, says envoy Lindner  https://t.co/5VXpoZgyhq via @ThePrintIndia #COVIDEmergency2021 #COVIDSecondWaveInIndia #CovidIndia',
 'TBC, this is an old report. The point is that Indian government is known to use national and international trolls to run propaganda campaigns or to cover up for its failures via domestic and international social media covert operations. #CovidIndiaInfo #COVIDindia2021 #CovidIndia',
 '"Most government-linked disinformation efforts were focused

In [14]:
Cov19_tweets['Tweet'] = Cov19_tweets['Tweet'].apply(text_process)

<ipython-input-14-42cb32dabff5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Cov19_tweets['Tweet'] = Cov19_tweets['Tweet'].apply(text_process)


In [15]:
Cov19_tweets

,Date_of_tweet,Tweet,User_handle
0,2021-04-27 23:55:06 UTC,varian baru covid india sudah masuk indonesia ...,23772644
1,2021-04-27 23:35:39 UTC,badai covid india hingga junta myanmar pikir p...,17128975
2,2021-04-27 23:18:00 UTC,peringatan who krisis covid india bisa makin b...,61377303
3,2021-04-27 23:16:27 UTC,german army work around the clock to help indi...,576348125
4,2021-04-27 22:04:08 UTC,tbc this be an old report the point be that in...,559652727
...,...,...,...
9650,2020-03-06 17:13:33 UTC,covid or no covid india be not china or usa we...,4888435970
9651,2020-03-04 05:36:35 UTC,watch coronavirusupdate from gandhi hospital i...,31059478
9652,2020-03-03 11:14:53 UTC,coronavirusupdate a member team from telangana...,31059478
9653,2020-03-02 12:39:20 UTC,ye gad now ndtv s report an italian tourist te...,73043514


In [16]:
tweets = Cov19_tweets['Tweet'].tolist()
tweets

['varian baru covid india sudah masuk indonesia dengan adanya orang yang terpapar enam orang kasus impor dan empat orang penularan lokal di sumatra dan jawa barat penelurusuran kontak indonesia lemah kata ahli penyakit menular',
 'badai covid india hingga junta myanmar pikir pikir usul asean kilasinternasional',
 'peringatan who krisis covid india bisa makin buruk',
 'german army work around the clock to help india with its covid crisis say envoy lindner via covidemergency covidsecondwaveinindia covidindia',
 'tbc this be an old report the point be that indian government be know to use national and international troll to run propaganda campaign or to cover up for its failure via domestic and international social medium covert operation covidindiainfo covidindia covidindia',
 'most government link disinformation effort be focus domestically researcher conclude but at least seven country have try to influence view outside their border china india iran pakistan russia saudi arabia and ven

## Get tri-grams from each tweet

In [17]:
def get_ngrams(sentence, n):
    words = sentence.split()
    phrases = []
    for i in range(len(words)-(n-1)):
        phrase = " ".join(words[i:i+n])
        phrases.append(phrase)
    return phrases

In [18]:
# get n-grams for each tweet
ngrams = []
for tweet in tweets:
    ngrams.append(get_ngrams(tweet, 3))

# flatten list of lists
ngrams = [item for sublist in ngrams for item in sublist]
print(len(ngrams))

# get unique n-grams
unique_ngrams = list(set(ngrams))
print(len(unique_ngrams))

142621
101455


## Sentence Transformer Models

In [19]:
# Initializing the sentence transformer models
xlm_dist = SentenceTransformer('sentence-transformers/xlm-r-distilroberta-base-paraphrase-v1')

distilbert = SentenceTransformer('sentence-transformers/msmarco-distilbert-base-tas-b')

para_minilm = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

## Getting the embeddings of tweets and tri-grams

In [20]:
xlm_embeddings = xlm_dist.encode(tweets, convert_to_tensor=True, show_progress_bar=True)
print("\nXLM-Roberta embeddings of tweets:")
print(xlm_embeddings)

Batches:   0%|          | 0/302 [00:00<?, ?it/s]


XLM-Roberta embeddings of tweets:
tensor([[-0.1752,  0.2369,  0.1251,  ...,  0.1741,  0.0560, -0.1252],
        [-0.1963, -0.0117,  0.1178,  ..., -0.1254,  0.1176,  0.1449],
        [-0.0610,  0.0258,  0.1859,  ..., -0.1053,  0.1619,  0.1157],
        ...,
        [-0.1860,  0.3181,  0.2821,  ...,  0.6127,  0.2973,  0.0051],
        [-0.2305,  0.1477,  0.4921,  ...,  0.7628,  0.0326, -0.1704],
        [-0.3148,  0.3153,  0.2703,  ...,  0.5160,  0.0102, -0.0432]],
       device='cuda:0')


In [21]:
distilbert_embeddings = distilbert.encode(tweets, convert_to_tensor=True, show_progress_bar=True)
print("\nDistilbert embeddings of tweets:")
print(distilbert_embeddings)

Batches:   0%|          | 0/302 [00:00<?, ?it/s]


Distilbert embeddings of tweets:
tensor([[-0.1794,  0.4257,  0.0686,  ...,  0.0240,  0.4532, -0.1039],
        [-0.0463,  0.2862, -0.0506,  ..., -0.3408, -0.0729, -0.2239],
        [-0.1425, -0.0101, -0.1924,  ..., -0.0987,  0.1697, -0.0773],
        ...,
        [ 0.2510, -0.1729,  0.2834,  ..., -0.2056,  0.1694,  0.1739],
        [ 0.3167, -0.2100,  0.2638,  ..., -0.4495,  0.0752,  0.2543],
        [ 0.4158, -0.2318,  0.3910,  ..., -0.0945,  0.2853,  0.1769]],
       device='cuda:0')


In [22]:
minilm_embeddings = para_minilm.encode(tweets, convert_to_tensor=True, show_progress_bar=True)
print("\nPara MiniLM embeddings of tweets:")
print(minilm_embeddings)

Batches:   0%|          | 0/302 [00:00<?, ?it/s]


Para MiniLM embeddings of tweets:
tensor([[ 0.0950,  0.4272,  0.0436,  ..., -0.2528,  0.0355,  0.5315],
        [-0.1484,  0.3785,  0.0525,  ..., -0.1367, -0.0895,  0.2901],
        [-0.2219,  0.2691, -0.6692,  ..., -0.3300,  0.0019,  0.2622],
        ...,
        [ 0.3213, -0.1629, -0.5856,  ..., -0.0245,  0.0219,  0.1144],
        [ 0.1647,  0.1621, -0.2067,  ..., -0.0426, -0.2348,  0.0771],
        [ 0.2491,  0.0986, -0.3194,  ..., -0.1592,  0.2886,  0.3092]],
       device='cuda:0')


In [23]:
xlm_ngram_embeddings = xlm_dist.encode(unique_ngrams, convert_to_tensor=True, show_progress_bar=True)
print("\nXLM-Roberta embeddings of n-grams:")
print(xlm_ngram_embeddings)

Batches:   0%|          | 0/3171 [00:00<?, ?it/s]


XLM-Roberta embeddings of n-grams:
tensor([[ 0.0517,  0.0937,  0.1309,  ..., -0.0282, -0.0940,  0.1848],
        [ 0.1599,  0.4967, -0.0178,  ...,  0.2965,  0.1592, -0.4724],
        [ 0.2373,  0.0369,  0.2050,  ...,  0.3041,  0.1021, -0.1123],
        ...,
        [-0.0665, -0.1044,  0.1253,  ...,  0.2816, -0.1569, -0.3087],
        [ 0.0182, -0.1278,  0.1647,  ...,  0.1364, -0.1422, -0.0611],
        [-0.1162, -0.0590,  0.5461,  ..., -0.0796,  0.1390, -0.0010]],
       device='cuda:0')


In [24]:
distilbert_ngram_embeddings = distilbert.encode(unique_ngrams, convert_to_tensor=True, show_progress_bar=True)
print("\nDistilbert embeddings of n-grams:")
print(distilbert_ngram_embeddings)

Batches:   0%|          | 0/3171 [00:00<?, ?it/s]


Distilbert embeddings of n-grams:
tensor([[-0.0067,  0.0136,  0.1875,  ..., -0.3914,  0.0507, -0.2280],
        [ 0.3692, -0.2223,  0.1915,  ...,  0.2396, -0.1822,  0.1598],
        [-0.3553, -0.2225, -0.3244,  ...,  0.2009, -0.2410, -0.2179],
        ...,
        [ 0.4169,  0.2139, -0.1342,  ..., -0.2409,  0.1553,  0.1897],
        [-0.0467,  0.0695,  0.2149,  ...,  0.1139,  0.0765,  0.5322],
        [-0.0536, -0.1621,  0.2777,  ..., -0.2778, -0.0640, -0.2156]],
       device='cuda:0')


In [25]:
minilm_ngram_embeddings = para_minilm.encode(unique_ngrams, convert_to_tensor=True, show_progress_bar=True)
print("\nPara MiniLM embeddings of n-grams:")
print(minilm_ngram_embeddings)

Batches:   0%|          | 0/3171 [00:00<?, ?it/s]


Para MiniLM embeddings of n-grams:
tensor([[-0.2046,  0.0478, -0.8342,  ...,  0.0175, -0.5616, -0.1521],
        [-0.0841,  0.2619,  0.0386,  ...,  0.0511,  0.2461,  0.2078],
        [ 0.3711,  0.6350, -0.5092,  ..., -0.4554,  0.3538, -0.1365],
        ...,
        [ 0.3515, -0.4144,  0.3433,  ..., -0.5283, -0.0555, -0.2616],
        [ 0.9477,  0.7245, -0.1327,  ...,  0.0840,  0.3189,  0.1694],
        [ 0.5899,  0.0764, -0.3608,  ..., -0.1213,  0.3905, -0.0757]],
       device='cuda:0')


In [26]:
torch.save(xlm_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/Covid19_data_embeddings/xlm_tweet_embeddings.pt")
torch.save(xlm_ngram_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/Covid19_data_embeddings/xlm_ngram_embeddings.pt")
torch.save(distilbert_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/Covid19_data_embeddings/distilbert_tweet_embeddings.pt")
torch.save(distilbert_ngram_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/Covid19_data_embeddings/distilbert_ngram_embeddings.pt")
torch.save(minilm_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/Covid19_data_embeddings/minilm_tweet_embeddings.pt")
torch.save(minilm_ngram_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/Covid19_data_embeddings/minilm_ngram_embeddings.pt")

## Computing average Cosine Similarity between each tri-gram and the document

In [27]:
xlm_similarities = []
for ngram in xlm_ngram_embeddings:
    xlm_similarities.append(util.pytorch_cos_sim(xlm_embeddings, ngram).mean().item())

In [28]:
xlm_similarities

[0.15044085681438446,
 0.09205146878957748,
 0.13070522248744965,
 0.07383280247449875,
 0.18644438683986664,
 0.11293929815292358,
 0.12001563608646393,
 0.08382701128721237,
 0.06399345397949219,
 0.07831156253814697,
 0.10837860405445099,
 0.1453004777431488,
 0.11599625647068024,
 0.15186327695846558,
 0.2222418338060379,
 0.1281319558620453,
 0.17747066915035248,
 0.11563719063997269,
 0.2249867022037506,
 0.14337828755378723,
 0.14201244711875916,
 0.12058158963918686,
 0.2807547450065613,
 0.15650169551372528,
 0.10074765980243683,
 0.10386903584003448,
 0.18925535678863525,
 0.29912468791007996,
 0.11891832202672958,
 0.11278427392244339,
 0.08541634678840637,
 0.14307726919651031,
 0.08698036521673203,
 0.10048233717679977,
 0.14093869924545288,
 0.13277265429496765,
 0.1294584572315216,
 0.05856303870677948,
 0.08290756493806839,
 0.024359023198485374,
 0.36379948258399963,
 0.12233638018369675,
 0.15699902176856995,
 0.14247120916843414,
 0.10259325057268143,
 0.233132988214

In [29]:
# minilm_similarity_matrix = util.pytorch_cos_sim(minilm_embeddings, minilm_ngram_embeddings)
# minilm_similarity_matrix

minilm_similarities = []
for ngram in minilm_ngram_embeddings:
    minilm_similarities.append(util.pytorch_cos_sim(minilm_embeddings, ngram).mean().item())

In [30]:
minilm_similarities

[0.08470513671636581,
 -0.04249086603522301,
 0.08969683945178986,
 -0.027837619185447693,
 0.12783533334732056,
 0.06757447123527527,
 0.08562563359737396,
 0.07918336987495422,
 0.028486425057053566,
 0.0711498036980629,
 0.07236543297767639,
 0.09106668829917908,
 0.14802150428295135,
 0.05036463215947151,
 0.20314963161945343,
 0.37378954887390137,
 0.07191991806030273,
 0.08176522701978683,
 0.3182612657546997,
 0.08836250752210617,
 0.07659375667572021,
 0.02600141055881977,
 0.18683291971683502,
 0.03021891973912716,
 0.04182863608002663,
 0.04526461288332939,
 0.2025524377822876,
 0.3274886906147003,
 0.020832126960158348,
 0.07717908173799515,
 0.008615449070930481,
 0.3312790095806122,
 0.06787735223770142,
 0.07365118712186813,
 0.08634603768587112,
 0.03691409155726433,
 0.08004273474216461,
 0.0053085824474692345,
 0.02501072734594345,
 0.09898513555526733,
 0.4293097257614136,
 0.04750519245862961,
 0.12759347259998322,
 0.13858014345169067,
 0.02255464345216751,
 0.16365

In [31]:
# minilm_similarity_matrix.shape

In [32]:
# # distilbert_similarity_matrix = util.pytorch_cos_sim(distilbert_embeddings, distilbert_ngram_embeddings)
# from sklearn.metrics.pairwise import cosine_similarity
# distilbert_embeddings = distilbert_embeddings.to("cpu")
# distilbert_ngram_embeddings = distilbert_ngram_embeddings.to("cpu")
# distilbert_similarity_matrix = util.pytorch_cos_sim(distilbert_embeddings, distilbert_ngram_embeddings)
# distilbert_similarity_matrix

distilbert_similarities = []
for ngram in distilbert_ngram_embeddings:
    distilbert_similarities.append(util.pytorch_cos_sim(distilbert_embeddings, ngram).mean().item())

In [33]:
# distilbert_similarity_matrix.shape

In [34]:
distilbert_similarities

[0.6260762810707092,
 0.6292790174484253,
 0.630073606967926,
 0.617132306098938,
 0.6494857668876648,
 0.6130375862121582,
 0.659650444984436,
 0.6318230032920837,
 0.6138675808906555,
 0.6465335488319397,
 0.635689377784729,
 0.6177207231521606,
 0.6107203364372253,
 0.6472733020782471,
 0.6627843976020813,
 0.7603492736816406,
 0.6789223551750183,
 0.6125765442848206,
 0.7560462951660156,
 0.652475118637085,
 0.6407487988471985,
 0.6893609762191772,
 0.736098051071167,
 0.5920864343643188,
 0.646034836769104,
 0.6330268383026123,
 0.6914675235748291,
 0.8110276460647583,
 0.6231462955474854,
 0.6866177320480347,
 0.6681544184684753,
 0.7645111083984375,
 0.6423763632774353,
 0.6642231941223145,
 0.614715576171875,
 0.633751392364502,
 0.6395912170410156,
 0.6621690988540649,
 0.6546885967254639,
 0.6302820444107056,
 0.805099368095398,
 0.5875145792961121,
 0.6092223525047302,
 0.6544886231422424,
 0.6251679062843323,
 0.6946136951446533,
 0.646194577217102,
 0.7038396596908569,
 0.

In [35]:
xlm_similarities = np.array(xlm_similarities)
distilbert_similarities = np.array(distilbert_similarities)
minilm_similarities = np.array(minilm_similarities)

## Normailizing the similarities using MinMaxScaler

In [36]:
scaler = MinMaxScaler()
xlm_similarities = scaler.fit_transform(xlm_similarities.reshape(-1, 1))
distilbert_similarities = scaler.fit_transform(distilbert_similarities.reshape(-1, 1))
minilm_similarities = scaler.fit_transform(minilm_similarities.reshape(-1, 1))

## Putting keyphrases and scores in a dataframe and ranking them individually for each model

In [37]:
xlm_scores_df = pd.DataFrame(list(zip(unique_ngrams, xlm_similarities)), columns=['keyphrase', 'score'])
distilbert_scores_df = pd.DataFrame(list(zip(unique_ngrams, distilbert_similarities)), columns=['keyphrase', 'score'])
minilm_scores_df = pd.DataFrame(list(zip(unique_ngrams, minilm_similarities)), columns=['keyphrase', 'score'])

In [38]:
xlm_scores_df

,keyphrase,score
0,germany to send,[0.3627367603598234]
1,float that have,[0.2546221798801665]
2,capital to combat,[0.32619399295061613]
3,be flat than,[0.2208882509560141]
4,disease filth undrinkable,[0.4294013834389005]
...,...,...
101450,family and friend,[0.2659310505503463]
101451,delhi record over,[0.3763780937968628]
101452,do not suggest,[0.2535805572157022]
101453,there be foreigner,[0.34438169257116275]


In [39]:
remove_list = []
threshold = 0.3

anomalies1 = xlm_scores_df[xlm_scores_df['score'] < threshold]['keyphrase'].tolist()
remove_list.append(anomalies1)

anomalies2 = distilbert_scores_df[distilbert_scores_df['score'] < threshold]['keyphrase'].tolist()
remove_list.append(anomalies2)

anomalies3 = minilm_scores_df[minilm_scores_df['score'] < threshold]['keyphrase'].tolist()
remove_list.append(anomalies3)

remove_list = [item for sublist in remove_list for item in sublist]
remove_list = list(set(remove_list))

len(remove_list)

58238

In [40]:
# removing the keyphrases from remove_list from all 3 dataframes
xlm_scores_df = xlm_scores_df[~xlm_scores_df['keyphrase'].isin(remove_list)]
distilbert_scores_df = distilbert_scores_df[~distilbert_scores_df['keyphrase'].isin(remove_list)]
minilm_scores_df = minilm_scores_df[~minilm_scores_df['keyphrase'].isin(remove_list)]

In [41]:
xlm_rank_df = xlm_scores_df.sort_values(by='score', ascending=False).reset_index(drop=True)
xlm_rank_df['rank'] = xlm_rank_df.index + 1
xlm_rank_df

,keyphrase,score,rank
0,covidindia coronavirusinindia delhi,[1.0],1
1,covidindia coronavirusindia covidemergency,[0.995228875969896],2
2,covidindia coronavirusindiaupdate covid,[0.9901858131555961],3
3,coronaupdatesinindia coronavirusoutbreakindia ...,[0.9896426529581855],4
4,covidindia coronavirusinindia coronaupdatesindia,[0.9896349274245436],5
...,...,...,...
43212,and globally more,[0.30002845203675554],43213
43213,and isolation bed,[0.3000184640254042],43214
43214,s covid response,[0.3000135252021117],43215
43215,daily dwindle by,[0.3000073171840066],43216


In [42]:
distilbert_rank_df = distilbert_scores_df.sort_values(by='score', ascending=False).reset_index(drop=True)
distilbert_rank_df['rank'] = distilbert_rank_df.index + 1
distilbert_rank_df

,keyphrase,score,rank
0,covid covidindia indiafightscoronavirus,[1.0],1
1,covidindia coronavirusindia indiafightscovid,[0.9992852584893721],2
2,covidindia covid indiafightscoronavirus,[0.9915495546879174],3
3,covidindia coronavirusoutbreak indiafightscoro...,[0.989894721425391],4
4,covid indiafightscoronavirus covidindia,[0.9885367691758615],5
...,...,...,...
43212,suffer from any,[0.3000642267061173],43213
43213,help the people,[0.3000408234984815],43214
43214,not collapse in,[0.300039879820754],43215
43215,say congress telegraph,[0.3000266683325723],43216


In [43]:
minilm_rank_df = minilm_scores_df.sort_values(by='score', ascending=False).reset_index(drop=True)
minilm_rank_df['rank'] = minilm_rank_df.index + 1
minilm_rank_df

,keyphrase,score,rank
0,covidindia coronavirusindia indiafightscovid,[1.0],1
1,covidindia coronavirusindiaupdate covid,[0.9938055754309639],2
2,covidindia covidhelp coronavirusindia,[0.9911478339734783],3
3,covidindia covidindia coronaviruspakistan,[0.9902772409634568],4
4,covid covidindia coronavirusreachesdelhi,[0.9887267272009191],5
...,...,...,...
43212,dawn on I,[0.3000546993282977],43213
43213,early hostel will,[0.30005151809095054],43214
43214,you donate donateplasmasavelive,[0.30004120010950186],43215
43215,flight between dubai,[0.30003186960207984],43216


In [44]:
# temp1 = xlm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
# temp2 = distilbert_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
# temp3 = minilm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)

# # create a dataframe with n-gram, average score
# avg_score_df = pd.DataFrame(list(zip(temp1['keyphrase'], (temp1['score'] + temp2['score'] + temp3['score'])/3)), columns=['keyphrase', 'avg_score'])
# avg_score_df

In [45]:
# avg_score_df = avg_score_df.sort_values(by='avg_score', ascending=False).reset_index(drop=True)
# avg_score_df

In [46]:
# Computing threshold

# avg_scores = avg_score_df['avg_score'].tolist()

# dk = 0
# for i in range(len(avg_scores)-1):
#     dk += abs(avg_scores[i+1] - avg_scores[i])

# dk = dk/(len(avg_scores)-1)

# sk = 0
# for i in range(len(avg_scores)-1):
#     sk += abs(abs(avg_scores[i+1] - avg_scores[i]) - dk)

# sk = sk/(len(avg_scores)-1)
# sk = np.sqrt(sk)

# threshold = sk/dk
# threshold = threshold[0]

# print("d(k) = ", dk)
# print("s(k) = ", sk)
# print("Threshold = ", threshold)

In [47]:
# threshold = 0.5

In [48]:
# anomalies = avg_score_df[avg_score_df['avg_score'] < threshold]['keyphrase'].tolist()
# anomalies

In [49]:
# remove these keyphrases from xlm_rank_df, distilbert_rank_df and minilm_rand_df
# xlm_rank_df = xlm_rank_df[~xlm_rank_df['keyphrase'].isin(anomalies)]
# distilbert_rank_df = distilbert_rank_df[~distilbert_rank_df['keyphrase'].isin(anomalies)]
# minilm_rank_df = minilm_rank_df[~minilm_rank_df['keyphrase'].isin(anomalies)]

## Processing to get the array to perform Rank Aggregation

In [50]:
# sort xlm_rank_df, distilbert_rank_df, minilm_rank_df by alphabetical order of n-grams
temp1 = xlm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
temp2 = distilbert_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
temp3 = minilm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)

# form a matrix where columns are the keyphrases and three rows, each row representing the rank of the keyphrase in each model
keyphrase_matrix = pd.DataFrame(columns=['xlm_rank', 'distilbert_rank', 'minilm_rank'])
keyphrase_matrix['xlm_rank'] = temp1['rank']
keyphrase_matrix['distilbert_rank'] = temp2['rank']
keyphrase_matrix['minilm_rank'] = temp3['rank']

m = keyphrase_matrix.to_numpy()
m = m.T
m

array([[34234,  8966, 13196, ..., 30062,  3250,  7731],
       [39434, 24824, 24955, ..., 22283, 11213,  9535],
       [35117, 12752, 25675, ..., 26376, 14760, 17997]])

In [51]:
m.shape

(3, 43217)

## Rank Aggregation using `ranky` to get the final rank for each keyphrase

In [52]:
# agg_ranks = rk.kemeny_young(m)
# agg_ranks

In [53]:
temp1 = xlm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
temp2 = distilbert_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
temp3 = minilm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)

# create a dataframe with n-gram, average score
avg_score_df = pd.DataFrame(list(zip(temp1['keyphrase'], (temp1['score'] * 0.45 + temp2['score'] * 0.3 + temp3['score'] * 0.25))), columns=['keyphrase', 'avg_score'])
avg_score_df = avg_score_df.sort_values(by='avg_score', ascending=False).reset_index(drop=True)
avg_score_df

,keyphrase,avg_score
0,covidindia coronavirusindia indiafightscovid,[0.9917974474981581]
1,covidindia coronavirusindiaupdate covid,[0.9833924199788021]
2,covid covidindia coronavirusoutbreakindia,[0.9797242636531908]
3,indiacovid covidindia coronavirusindia,[0.9792561989479057]
4,covidindia coronavirusindia covidemergency,[0.9788871712975942]
...,...,...
43212,out by authority,[0.3047779745445379]
43213,our helpline number,[0.3044394410185936]
43214,the come decade,[0.304192604845547]
43215,expected but now,[0.30417004442948614]


In [54]:
final_rank_df = avg_score_df.sort_values(by='avg_score', ascending=False).reset_index(drop=True)
final_rank_df['rank'] = final_rank_df.index + 1
final_rank_df

,keyphrase,avg_score,rank
0,covidindia coronavirusindia indiafightscovid,[0.9917974474981581],1
1,covidindia coronavirusindiaupdate covid,[0.9833924199788021],2
2,covid covidindia coronavirusoutbreakindia,[0.9797242636531908],3
3,indiacovid covidindia coronavirusindia,[0.9792561989479057],4
4,covidindia coronavirusindia covidemergency,[0.9788871712975942],5
...,...,...,...
43212,out by authority,[0.3047779745445379],43213
43213,our helpline number,[0.3044394410185936],43214
43214,the come decade,[0.304192604845547],43215
43215,expected but now,[0.30417004442948614],43216


In [55]:
final_rank_df.head(20)

,keyphrase,avg_score,rank
0,covidindia coronavirusindia indiafightscovid,[0.9917974474981581],1
1,covidindia coronavirusindiaupdate covid,[0.9833924199788021],2
2,covid covidindia coronavirusoutbreakindia,[0.9797242636531908],3
3,indiacovid covidindia coronavirusindia,[0.9792561989479057],4
4,covidindia coronavirusindia covidemergency,[0.9788871712975942],5
5,covid covidindia coronavirusreachesdelhi,[0.9769089104102432],6
6,indiafightscovid coronavirusindia covidindia,[0.9768124345329983],7
7,covidindia covidhelp coronavirusindia,[0.9741239201973018],8
8,covidindia coronavirusoutbreakindia rbigovernor,[0.9737586500480978],9
9,covidindia covidemergency coronavirusindia,[0.9729058464572096],10


In [56]:
concatenated_tweets = ' '.join(raw_tweets)
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(concatenated_tweets, keyphrase_ngram_range=(3, 3), top_n=100)
keywords

[('indian government covidindia', 0.6917),
 ('covid india campaign', 0.6841),
 ('government india covid', 0.6835),
 ('india reports covid', 0.6805),
 ('covid india reports', 0.677),
 ('reporting covid india', 0.6751),
 ('india reporting covidindia', 0.6692),
 ('covidupdate india reported', 0.669),
 ('covidemergency covidindia india_narrative', 0.6662),
 ('covid19 india reports', 0.6638),
 ('covid india govt', 0.6638),
 ('india covidindia cnnnews18', 0.6635),
 ('india tenterhooks covidindia', 0.6628),
 ('cnnnews18 covid india', 0.6621),
 ('india current covid', 0.6596),
 ('indian govt covidnewstrain', 0.6589),
 ('covid india government', 0.6568),
 ('combating covid india', 0.6544),
 ('covidindia india government', 0.654),
 ('covidindia report indian', 0.6524),
 ('covid19 india posts', 0.6519),
 ('responding covid india', 0.6499),
 ('covidindia india efforts', 0.6498),
 ('india covid covidupdates', 0.6496),
 ('india covid situation', 0.6494),
 ('reality covid india', 0.6489),
 ('covid19 

In [57]:
ground_truth = [x[0] for x in keywords]
ground_truth

['indian government covidindia',
 'covid india campaign',
 'government india covid',
 'india reports covid',
 'covid india reports',
 'reporting covid india',
 'india reporting covidindia',
 'covidupdate india reported',
 'covidemergency covidindia india_narrative',
 'covid19 india reports',
 'covid india govt',
 'india covidindia cnnnews18',
 'india tenterhooks covidindia',
 'cnnnews18 covid india',
 'india current covid',
 'indian govt covidnewstrain',
 'covid india government',
 'combating covid india',
 'covidindia india government',
 'covidindia report indian',
 'covid19 india posts',
 'responding covid india',
 'covidindia india efforts',
 'india covid covidupdates',
 'india covid situation',
 'reality covid india',
 'covid19 india reported',
 'covid india policy',
 'brief india covid',
 'covid19 india govt',
 'regime covidemergency2021 bjpdestroyedindia',
 'india covidindia noting',
 'fakenews covid india',
 'help india covid',
 'covidindia india reports',
 'support india covid'

In [58]:
precision = 0

for keyphrase in final_rank_df.head(100)['keyphrase'].tolist():
    if keyphrase in ground_truth:
        precision += 1

print("Proposed Model Precision: ", precision)

Proposed Model Precision:  0


In [59]:
recall = 0

for keyphrase in ground_truth:
    if keyphrase in final_rank_df.head(100)['keyphrase'].tolist():
        recall += 1

print("Proposed Model Recall: ", recall)

Proposed Model Recall:  0


In [60]:
xlm_precision = 0
for keyphrase in xlm_rank_df.head(100)['keyphrase'].tolist():
    if keyphrase in ground_truth:
        xlm_precision += 1

print("xlm-roberto Precision: ", xlm_precision)

distilbert_precision = 0
for keyphrase in distilbert_rank_df.head(100)['keyphrase'].tolist():
    if keyphrase in ground_truth:
        distilbert_precision += 1

print("Distilbert Precision: ", distilbert_precision)

minilm_precision = 0
for keyphrase in minilm_rank_df.head(100)['keyphrase'].tolist():
    if keyphrase in ground_truth:
        minilm_precision += 1

print("Mini LM Precision: ", minilm_precision)

xlm-roberto Precision:  0
Distilbert Precision:  0
Mini LM Precision:  0


In [61]:
xlm_recall = 0
for keyphrase in ground_truth:
    if keyphrase in xlm_rank_df.head(100)['keyphrase'].tolist():
        xlm_recall += 1

print("xlm-roberto Recall: ", xlm_recall)

distilbert_recall = 0
for keyphrase in ground_truth:
    if keyphrase in distilbert_rank_df.head(100)['keyphrase'].tolist():
        distilbert_recall += 1

print("distilbert Recall: ", distilbert_recall)

minilm_recall = 0
for keyphrase in ground_truth:
    if keyphrase in minilm_rank_df.head(100)['keyphrase'].tolist():
        minilm_recall += 1

print("Mini LM Recall: ", minilm_recall)

xlm-roberto Recall:  0
distilbert Recall:  0
Mini LM Recall:  0


In [62]:
import scipy.stats

def rank_correlation(rank1, rank2):
  
    rho, pval = scipy.stats.spearmanr(rank1, rank2)
    return rho

# model_ranking = ["keyphrase1", "keyphrase2", "keyphrase3"]
# ground_truth = ["keyphrase2", "keyphrase1", "keyphrase4"]
similarity = rank_correlation(final_rank_df.head(100)['keyphrase'].tolist(), ground_truth)
print("spearman similarity of proposed model with KeyBERT: ", similarity)

spearman similarity of proposed model with KeyBERT:  0.06546654665466546


In [63]:
xlm_similarity = rank_correlation(xlm_rank_df.head(100)['keyphrase'].tolist(), ground_truth)
print("spearman similarity of xlm-roberto with KeyBERT: ", xlm_similarity)

distilbert_similarity = rank_correlation(distilbert_rank_df.head(100)['keyphrase'].tolist(), ground_truth)
print("spearman similarity of distilbert model with KeyBERT: ", distilbert_similarity)

minilm_similarity = rank_correlation(minilm_rank_df.head(100)['keyphrase'].tolist(), ground_truth)
print("spearman similarity of Mini LM model with KeyBERT: ", minilm_similarity)

spearman similarity of xlm-roberto with KeyBERT:  -0.20638463846384636
spearman similarity of distilbert model with KeyBERT:  0.05327332733273327
spearman similarity of Mini LM model with KeyBERT:  -0.02117011701170117


In [64]:
# def get_embeddings(sentence):    
    
#     marked_sentence = "[CLS] " + sentence + " [SEP]"
#     tokenized_sentence = tokenizer.tokenize(marked_sentence)
#     indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_sentence)
#     segment_ids = [1] * len(tokenized_sentence)

#     tokens_tensor1 = torch.tensor([indexed_tokens]) #input_ids
#     #tokens_tensor2 = tf.Tensor([indexed_tokens])

#     segment_tensors1 = torch.tensor([segment_ids]) #attention_mask
#     #segment_tensors2 = tf.Tensor([segment_ids])

#     with torch.no_grad():
#         outputs = model(tokens_tensor1, segment_tensors1)
    
#     print(outputs.keys())
    
#     hidden_states = outputs[1]
#     left_tensor = outputs.last_hidden_state
#     cls_tensor = left_tensor[:, 0, :]
#     token_embeddings = torch.stack(hidden_states, dim=0)
#     token_embeddings.size()
#     token_embeddings = torch.squeeze(token_embeddings, dim=1)

#     token_embeddings = token_embeddings.permute(1,0,2)
    
#     token_vecs_list = []

#     for token in token_embeddings:
#         sum_vec = torch.sum(token[-4:], dim=0)
#         token_vecs_list.append(sum_vec)
    
#     return token_vecs_list